In [16]:
#read file
mylist = []
with open("28.txt") as f:
    mylist = f.read().splitlines()
str1 = ''.join(mylist)
mylist = []
with open("ans.txt") as f:
    mylist = f.read().splitlines()
str2 = ''.join(mylist)

for i in range(0, len(str1)):
    if str1[i]!=str2[i]:
        print "eror"
print len(str1)
print len(str2)

139812
139812


In [7]:
print sys.argv

['C:\\Users\\lckung\\Anaconda3\\envs\\ipykernel_py2\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\lckung\\AppData\\Roaming\\jupyter\\runtime\\kernel-2727178b-abe8-447c-bd62-a6a7daa02c0a.json']


In [9]:
import sys

class PorterStemmer:

    def __init__(self):
        """The main part of the stemming algorithm starts here.
        b is a buffer holding a word to be stemmed. The letters are in b[k0],
        b[k0+1] ... ending at b[k]. In fact k0 = 0 in this demo program. k is
        readjusted downwards as the stemming progresses. Zero termination is
        not in fact used in the algorithm.

        Note that only lower case sequences are stemmed. Forcing to lower case
        should be done before stem(...) is called.
        """

        self.b = ""  # buffer for word to be stemmed
        self.k = 0
        self.k0 = 0
        self.j = 0   # j is a general offset into the string

    def cons(self, i):
        """cons(i) is TRUE <=> b[i] is a consonant."""
        if self.b[i] == 'a' or self.b[i] == 'e' or self.b[i] == 'i' or self.b[i] == 'o' or self.b[i] == 'u':
            return 0
        if self.b[i] == 'y':
            if i == self.k0:
                return 1
            else:
                return (not self.cons(i - 1))
        return 1

    def m(self):
        """m() measures the number of consonant sequences between k0 and j.
        if c is a consonant sequence and v a vowel sequence, and <..>
        indicates arbitrary presence,

           <c><v>       gives 0
           <c>vc<v>     gives 1
           <c>vcvc<v>   gives 2
           <c>vcvcvc<v> gives 3
           ....
        """
        n = 0
        i = self.k0
        while 1:
            if i > self.j:
                return n
            if not self.cons(i):
                break
            i = i + 1
        i = i + 1
        while 1:
            while 1:
                if i > self.j:
                    return n
                if self.cons(i):
                    break
                i = i + 1
            i = i + 1
            n = n + 1
            while 1:
                if i > self.j:
                    return n
                if not self.cons(i):
                    break
                i = i + 1
            i = i + 1

    def vowelinstem(self):
        """vowelinstem() is TRUE <=> k0,...j contains a vowel"""
        for i in range(self.k0, self.j + 1):
            if not self.cons(i):
                return 1
        return 0

    def doublec(self, j):
        """doublec(j) is TRUE <=> j,(j-1) contain a double consonant."""
        if j < (self.k0 + 1):
            return 0
        if (self.b[j] != self.b[j-1]):
            return 0
        return self.cons(j)

    def cvc(self, i):
        """cvc(i) is TRUE <=> i-2,i-1,i has the form consonant - vowel - consonant
        and also if the second c is not w,x or y. this is used when trying to
        restore an e at the end of a short  e.g.

           cav(e), lov(e), hop(e), crim(e), but
           snow, box, tray.
        """
        if i < (self.k0 + 2) or not self.cons(i) or self.cons(i-1) or not self.cons(i-2):
            return 0
        ch = self.b[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends(self, s):
        """ends(s) is TRUE <=> k0,...k ends with the string s."""
        length = len(s)
        if s[length - 1] != self.b[self.k]: # tiny speed-up
            return 0
        if length > (self.k - self.k0 + 1):
            return 0
        if self.b[self.k-length+1:self.k+1] != s:
            return 0
        self.j = self.k - length
        return 1

    def setto(self, s):
        """setto(s) sets (j+1),...k to the characters in the string s, readjusting k."""
        length = len(s)
        self.b = self.b[:self.j+1] + s + self.b[self.j+length+1:]
        self.k = self.j + length

    def r(self, s):
        """r(s) is used further down."""
        if self.m() > 0:
            self.setto(s)

    def step1ab(self):
        """step1ab() gets rid of plurals and -ed or -ing. e.g.

           caresses  ->  caress
           ponies    ->  poni
           ties      ->  ti
           caress    ->  caress
           cats      ->  cat

           feed      ->  feed
           agreed    ->  agree
           disabled  ->  disable

           matting   ->  mat
           mating    ->  mate
           meeting   ->  meet
           milling   ->  mill
           messing   ->  mess

           meetings  ->  meet
        """
        if self.b[self.k] == 's':
            if self.ends("sses"):
                self.k = self.k - 2
            elif self.ends("ies"):
                self.setto("i")
            elif self.b[self.k - 1] != 's':
                self.k = self.k - 1
        if self.ends("eed"):
            if self.m() > 0:
                self.k = self.k - 1
        elif (self.ends("ed") or self.ends("ing")) and self.vowelinstem():
            self.k = self.j
            if self.ends("at"):   self.setto("ate")
            elif self.ends("bl"): self.setto("ble")
            elif self.ends("iz"): self.setto("ize")
            elif self.doublec(self.k):
                self.k = self.k - 1
                ch = self.b[self.k]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.k = self.k + 1
            elif (self.m() == 1 and self.cvc(self.k)):
                self.setto("e")

    def step1c(self):
        """step1c() turns terminal y to i when there is another vowel in the stem."""
        if (self.ends("y") and self.vowelinstem()):
            self.b = self.b[:self.k] + 'i' + self.b[self.k+1:]

    def step2(self):
        """step2() maps double suffices to single ones.
        so -ization ( = -ize plus -ation) maps to -ize etc. note that the
        string before the suffix must give m() > 0.
        """
        if self.b[self.k - 1] == 'a':
            if self.ends("ational"):   self.r("ate")
            elif self.ends("tional"):  self.r("tion")
        elif self.b[self.k - 1] == 'c':
            if self.ends("enci"):      self.r("ence")
            elif self.ends("anci"):    self.r("ance")
        elif self.b[self.k - 1] == 'e':
            if self.ends("izer"):      self.r("ize")
        elif self.b[self.k - 1] == 'l':
            if self.ends("bli"):       self.r("ble") # --DEPARTURE--
            # To match the published algorithm, replace this phrase with
            #   if self.ends("abli"):      self.r("able")
            elif self.ends("alli"):    self.r("al")
            elif self.ends("entli"):   self.r("ent")
            elif self.ends("eli"):     self.r("e")
            elif self.ends("ousli"):   self.r("ous")
        elif self.b[self.k - 1] == 'o':
            if self.ends("ization"):   self.r("ize")
            elif self.ends("ation"):   self.r("ate")
            elif self.ends("ator"):    self.r("ate")
        elif self.b[self.k - 1] == 's':
            if self.ends("alism"):     self.r("al")
            elif self.ends("iveness"): self.r("ive")
            elif self.ends("fulness"): self.r("ful")
            elif self.ends("ousness"): self.r("ous")
        elif self.b[self.k - 1] == 't':
            if self.ends("aliti"):     self.r("al")
            elif self.ends("iviti"):   self.r("ive")
            elif self.ends("biliti"):  self.r("ble")
        elif self.b[self.k - 1] == 'g': # --DEPARTURE--
            if self.ends("logi"):      self.r("log")
        # To match the published algorithm, delete this phrase

    def step3(self):
        """step3() dels with -ic-, -full, -ness etc. similar strategy to step2."""
        if self.b[self.k] == 'e':
            if self.ends("icate"):     self.r("ic")
            elif self.ends("ative"):   self.r("")
            elif self.ends("alize"):   self.r("al")
        elif self.b[self.k] == 'i':
            if self.ends("iciti"):     self.r("ic")
        elif self.b[self.k] == 'l':
            if self.ends("ical"):      self.r("ic")
            elif self.ends("ful"):     self.r("")
        elif self.b[self.k] == 's':
            if self.ends("ness"):      self.r("")

    def step4(self):
        """step4() takes off -ant, -ence etc., in context <c>vcvc<v>."""
        if self.b[self.k - 1] == 'a':
            if self.ends("al"): pass
            else: return
        elif self.b[self.k - 1] == 'c':
            if self.ends("ance"): pass
            elif self.ends("ence"): pass
            else: return
        elif self.b[self.k - 1] == 'e':
            if self.ends("er"): pass
            else: return
        elif self.b[self.k - 1] == 'i':
            if self.ends("ic"): pass
            else: return
        elif self.b[self.k - 1] == 'l':
            if self.ends("able"): pass
            elif self.ends("ible"): pass
            else: return
        elif self.b[self.k - 1] == 'n':
            if self.ends("ant"): pass
            elif self.ends("ement"): pass
            elif self.ends("ment"): pass
            elif self.ends("ent"): pass
            else: return
        elif self.b[self.k - 1] == 'o':
            if self.ends("ion") and (self.b[self.j] == 's' or self.b[self.j] == 't'): pass
            elif self.ends("ou"): pass
            # takes care of -ous
            else: return
        elif self.b[self.k - 1] == 's':
            if self.ends("ism"): pass
            else: return
        elif self.b[self.k - 1] == 't':
            if self.ends("ate"): pass
            elif self.ends("iti"): pass
            else: return
        elif self.b[self.k - 1] == 'u':
            if self.ends("ous"): pass
            else: return
        elif self.b[self.k - 1] == 'v':
            if self.ends("ive"): pass
            else: return
        elif self.b[self.k - 1] == 'z':
            if self.ends("ize"): pass
            else: return
        else:
            return
        if self.m() > 1:
            self.k = self.j

    def step5(self):
        """step5() removes a final -e if m() > 1, and changes -ll to -l if
        m() > 1.
        """
        self.j = self.k
        if self.b[self.k] == 'e':
            a = self.m()
            if a > 1 or (a == 1 and not self.cvc(self.k-1)):
                self.k = self.k - 1
        if self.b[self.k] == 'l' and self.doublec(self.k) and self.m() > 1:
            self.k = self.k -1

    def stem(self, p, i, j):
        """In stem(p,i,j), p is a char pointer, and the string to be stemmed
        is from p[i] to p[j] inclusive. Typically i is zero and j is the
        offset to the last character of a string, (p[j+1] == '\0'). The
        stemmer adjusts the characters p[i] ... p[j] and returns the new
        end-point of the string, k. Stemming never increases word length, so
        i <= k <= j. To turn the stemmer into a module, declare 'stem' as
        extern, and delete the remainder of this file.
        """
        # copy the parameters into statics
        self.b = p
        self.k = j
        self.k0 = i
        if self.k <= self.k0 + 1:
            return self.b # --DEPARTURE--

        # With this line, strings of length 1 or 2 don't go through the
        # stemming process, although no mention is made of this in the
        # published algorithm. Remove the line to match the published
        # algorithm.

        self.step1ab()
        self.step1c()
        self.step2()
        self.step3()
        self.step4()
        self.step5()
        return self.b[self.k0:self.k+1]


if __name__ == '__main__':
    p = PorterStemmer()
    if len(sys.argv) > 1:
        for f in sys.argv[1:]:
            infile = open("voc.txt", 'r')
            while 1:
                output = ''
                word = ''
                line = infile.readline()
                if line == '':
                    break
                for c in line:
                    if c.isalpha():
                        word += c.lower()
                    else:
                        if word:
                            output += p.stem(word, 0,len(word)-1)
                            word = ''
                        output += c.lower()
                print output,
            infile.close()

a
aaron
abaissiez
abandon
abandon
abas
abash
abat
abat
abat
abat
abat
abbess
abbei
abbei
abbomin
abbot
abbot
abbrevi
ab
abel
aberga
abergavenni
abet
abet
abhomin
abhor
abhorr
abhor
abhor
abhor
abhorson
abid
abid
abil
abil
abject
abjectli
abject
abjur
abjur
abl
abler
aboard
abod
abod
abod
abod
abomin
abomin
abomin
abort
abort
abound
abound
about
abov
abr
abraham
abram
abreast
abridg
abridg
abridg
abridg
abroach
abroad
abrog
abrook
abrupt
abrupt
abruptli
absenc
absent
absei
absolut
absolut
absolv
absolv
abstain
abstemi
abstin
abstract
absurd
absyrtu
abund
abund
abundantli
abu
abus
abus
abus
abus
abus
abut
abi
abysm
ac
academ
academ
accent
accent
accept
accept
accept
accept
accept
access
accessari
access
accid
accid
accident
accident
accid
accit
accit
accit
acclam
accommod
accommod
accommod
accommod
accommodo
accompani
accompani
accompani
accomplic
accomplish
accomplish
accomplish
accomplish
accompt
accord
accord
accord
accordeth
accord
accordingli
accord
accost
accost
account
account
acc

bonnet
bonnet
bonni
bono
bonto
bonvil
bood
book
bookish
book
boon
boor
boorish
boor
boot
boot
booti
bootless
boot
booti
bor
bora
borachio
bordeaux
border
border
border
border
bore
borea
bore
bore
born
born
borough
borough
borrow
borrow
borrow
borrow
borrow
bosko
bosko
boski
bosom
bosom
boson
boss
bosworth
botch
botcher
botch
botchi
both
bot
bottl
bottl
bottl
bottom
bottomless
bottom
bouciqualt
boug
bough
bough
bought
bounc
bounc
bound
bound
bounden
boundeth
bound
boundless
bound
bounteou
bounteous
bounti
bounti
bountifulli
bounti
bourbier
bourbon
bourchier
bourdeaux
bourn
bout
bout
bove
bow
bowcas
bow
bowel
bower
bow
bowl
bowler
bowl
bowl
bow
bowsprit
bowstr
box
box
boi
boyet
boyish
boi
brabant
brabantio
brabbl
brabbler
brac
brace
bracelet
bracelet
brach
braci
brag
bragg
braggard
braggard
braggart
braggart
brag
brag
bragless
brag
braid
braid
brain
brain
brainford
brainish
brainless
brain
brainsick
brainsickli
brake
brakenburi
brake
brambl
bran
branch
branch
branchless
brand
brand
brand

design
desir
desir
desir
desir
desir
desirest
desir
desir
desist
desk
desol
desol
desp
despair
despair
despair
despatch
desper
desper
desper
despi
despis
despis
despis
despiseth
despis
despit
despit
despoil
dest
destin
destin
destini
destini
destitut
destroi
destroi
destroy
destroy
destroi
destroi
destruct
destruct
det
detain
detain
detect
detect
detect
detect
detector
detect
detent
determin
determin
determin
determin
determin
determin
determin
detest
detest
detest
detest
detest
detract
detract
detract
deucalion
deuc
deum
deux
devant
devest
devic
devic
devil
devilish
devil
devi
devis
devis
devis
devis
devoid
devonshir
devot
devot
devot
devour
devour
devour
devour
devour
devout
devoutli
dew
dewberri
dewdrop
dewlap
dewlapp
dew
dewi
dexter
dexteri
dexter
di
diabl
diablo
diadem
dial
dialect
dialogu
dialogu
dial
diamet
diamond
diamond
dian
diana
diaper
dibbl
dic
dice
dicer
dich
dick
dicken
dickon
dicki
dictat
diction
dictynna
did
diddl
didest
dido
didst
die
di
diedst
di
diest
diet
diet
diet

greet
greet
greet
greet
greet
greg
gregori
gremio
grew
grei
greybeard
greybeard
greyhound
greyhound
grief
grief
griev
grievanc
grievanc
griev
griev
griev
grievest
griev
grievingli
grievou
grievous
griffin
griffith
grim
grime
grimli
grin
grind
grind
grindston
grin
grip
gripe
gripe
gripe
grise
grisli
grissel
grize
grizzl
grizzl
groan
groan
groan
groat
groat
groin
groom
groom
grop
grope
gro
gross
grosser
grossli
gross
ground
ground
groundl
ground
grove
grovel
grovel
grove
grow
groweth
grow
grown
grow
growth
grub
grubb
grub
grudg
grudg
grudg
grudg
gruel
grumbl
grumblest
grumbl
grumbl
grumio
grund
grunt
gualtier
guard
guardag
guardant
guard
guardian
guardian
guard
guardsman
gud
gudgeon
guerdon
guerra
guess
guess
guessingli
guest
guest
guiana
guichard
guid
guid
guider
guideriu
guid
guid
guidon
guienn
guil
guildenstern
guilder
guildford
guildhal
guil
guil
guil
guilford
guilt
guiltian
guiltier
guiltili
guilti
guiltless
guilt
guilti
guinea
guinev
guis
gul
gule
gulf
gulf
gull
gull
gum
gumm
gum
g

mourn
mourn
mourner
mourner
mourn
mournfulli
mourn
mourningli
mourn
mourn
mou
mous
mousetrap
mous
mouth
mouth
mouth
mov
movabl
move
moveabl
moveabl
move
mover
mover
move
moveth
move
movingli
movousu
mow
mowbrai
mower
mow
mow
moi
moi
moys
mr
much
muck
mud
mud
muddi
muddi
muffin
muffl
muffl
muffl
muffler
muffl
mugger
mug
mulberri
mulberri
mule
mule
mulet
mulier
mulier
muliteu
mull
mulmutiu
multipli
multipli
multipli
multipot
multitud
multitud
multitudin
mum
mumbl
mumbl
mummer
mummi
mun
munch
muniment
munit
murd
murder
murder
murder
murder
murder
murder
murder
mure
murk
murkiest
murki
murmur
murmur
murmur
murrain
murrai
murrion
murther
murther
murther
murther
murther
murther
mu
muscadel
muscovit
muscovit
muscovi
muse
muse
mush
mushroom
music
music
musician
musician
music
muse
muse
musk
musket
musket
musko
muss
mussel
mussel
must
mustachio
mustard
mustardse
muster
muster
muster
musti
mutabl
mutabl
mutat
mutat
mute
mute
mutest
mutin
mutin
mutin
mutin
mutini
mutin
mutini
mutiu
mutter
mutter


...

demur
demur
demur
den
denai
deni
denial
denial
deni
denier
deni
deniest
deni
denmark
denni
denni
denot
denot
denot
denounc
denounc
denounc
den
denunci
deni
deni
deo
depart
depart
departest
depart
departur
depech
depend
depend
depend
depend
depend
depend
depend
depend
depend
depend
depend
depend
deplor
deplor
depopul
depo
depos
depos
depos
depositari
deprav
deprav
deprav
deprav
deprav
depress
depriv
depriv
depth
depth
deput
deput
deput
deputi
deput
deputi
deracin
derbi
derceta
dere
derid
deris
deriv
deriv
deriv
deriv
deriv
deriv
derog
derog
derog
de
desartless
descant
descend
descend
descend
descend
descens
descent
descent
describ
describ
describ
descri
descript
descript
descri
desdemon
desdemona
desert
desert
deserv
deserv
deserv
deservedli
deserv
deserv
deserv
deservest
deserv
deserv
design
design
design
design
desir
desir
desir
desir
desir
desirest
desir
desir
desist
desk
desol
desol
desp
despair
despair
despair
despatch
desper
desper
desper
despi
despis
despis
despis
despiseth
despi

harshli
harsh
hart
hart
harum
harvest
ha
hast
hast
hast
hasten
hast
hastili
hast
hast
hasti
hat
hatch
hatch
hatchet
hatch
hatchment
hate
hate
hate
hater
hater
hate
hateth
hatfield
hath
hate
hatr
hat
haud
hauf
haught
haughti
haughti
haunch
haunch
haunt
haunt
haunt
haunt
hautboi
hautboi
have
haven
haven
haver
have
have
havior
haviour
havoc
hawk
hawk
hawk
hawthorn
hawthorn
hai
hazard
hazard
hazard
hazel
hazelnut
he
head
headborough
head
headier
head
headland
headless
headlong
head
headsman
headstrong
headi
heal
heal
heal
heal
health
health
health
healthsom
healthi
heap
heap
heap
hear
heard
hearer
hearer
hearest
heareth
hear
hear
heark
hearken
hearken
hear
hearsai
hears
hears
hearst
heart
heartach
heartbreak
heartbreak
heart
hearten
hearth
hearth
heartili
hearti
heartless
heartl
heartli
heart
heartsick
heartstr
hearti
heat
heat
heath
heathen
heathenish
heat
heat
heauti
heav
heav
heav
heaven
heavenli
heaven
heav
heavier
heaviest
heavili
heavi
heav
heav
heavi
hebona
hebrew
hecat
hectic
hecto

obei
obei
obei
obei
obidicut
object
object
object
object
oblat
oblat
oblig
oblig
oblig
obliqu
oblivion
oblivi
obloqui
obscen
obscen
obscur
obscur
obscur
obscur
obscur
obscur
obscur
obsequi
obsequi
obsequi
observ
observ
observ
observ
observ
observ
observ
observ
observ
observ
observ
observ
observingli
obsqu
obstacl
obstacl
obstinaci
obstin
obstin
obstruct
obstruct
obstruct
obtain
obtain
obtain
occas
occas
occid
occident
occult
occupat
occup
occup
occupi
occupi
occupi
occurr
occurr
occurr
ocean
ocean
octavia
octaviu
ocular
od
odd
oddest
oddli
odd
od
od
odiou
odorifer
odor
odour
odour
od
oeillad
oe
oeuvr
of
ofephesu
off
offal
offenc
offenc
offenc
offend
offend
offendendo
offend
offend
offendeth
offend
offendress
offend
offens
offenseless
offens
offens
offer
offer
offer
offer
offer
offert
offic
offic
offic
offic
offic
offic
offici
offici
offspr
oft
often
often
oftentim
oh
oil
oil
oili
old
oldcastl
olden
older
oldest
old
oliv
oliv
oliv
oliv
olivia
olympian
olympu
oman
oman
omen
omin
omiss
om

sicken
sicker
sickl
sicklemen
sickli
sickli
sickli
sick
sicl
sicyon
side
side
side
sieg
sieg
sienna
si
siev
sift
sift
sigeia
sigh
sigh
sigh
sigh
sight
sight
sightless
sightli
sight
sign
signal
signet
signieur
signific
signific
signifi
signifi
signifi
signifi
signior
signiori
signior
signiori
signor
signori
sign
signum
silenc
silenc
silenc
silenc
silent
silent
siliu
silk
silken
silkman
silk
silliest
silli
sill
silli
silva
silver
silver
silverli
silvia
silviu
sima
simil
simil
simoi
simon
simoni
simp
simpcox
simpl
simpl
simpler
simpl
simplic
simpli
simular
simul
sin
sinc
sincer
sincer
sincer
sinel
sinew
sinew
sinew
sinewi
sin
sinfulli
sing
sing
sing
singer
sing
singeth
sing
singl
singl
singl
singli
sing
singular
singularit
singular
singular
singul
sinist
sink
sink
sink
sinn
sinner
sinner
sin
sinon
sin
sip
sip
sir
sire
siren
sirrah
sir
sist
sister
sisterhood
sisterli
sister
sit
sith
sithenc
sit
sit
situat
situat
situat
siward
six
sixpenc
sixpenc
sixpenni
sixteen
sixth
sixti
siz
size
size
s

wake
wake
wakefield
waken
waken
wake
wakest
wake
wale
walk
walk
walk
walk
wall
wall
wallet
wallet
wallon
walloon
wallow
wall
walnut
walter
wan
wand
wander
wander
wander
wander
wander
wand
wane
wane
wane
wane
wann
want
want
wanteth
want
wanton
wantonli
wanton
wanton
want
wappen
war
warbl
warbl
ward
ward
warden
warder
warder
wardrob
wardrop
ward
ware
ware
warili
warkworth
warlik
warm
warm
warmer
warm
warm
warmth
warn
warn
warn
warn
warn
warp
warp
warr
warrant
warrant
warranteth
warrantis
warrant
warrant
warranti
warren
warren
war
warrior
warrior
war
wart
warwick
warwickshir
wari
wa
wash
wash
washer
wash
washford
wash
wasp
waspish
wasp
wassail
wassail
wast
wast
wast
wast
waster
wast
wast
wat
watch
watch
watcher
watch
watch
watch
watch
watchman
watchmen
watchword
water
waterdrop
water
waterfli
waterford
water
waterish
waterpot
waterrug
water
waterton
wateri
wav
wave
wave
waver
waver
waver
wave
wave
waw
wawl
wax
wax
waxen
wax
wax
wai
waylaid
waylai
wai
wayward
wayward
wayward
we
weak
weaken